<a href="https://colab.research.google.com/github/georgian-io/constrained_optimization_workshop/blob/main/Constrained_Optimization_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install --upgrade --user ortools

Requirement already up-to-date: ortools in /root/.local/lib/python3.7/site-packages (8.2.8710)


In [2]:
import numpy as np
import pandas as pd

from ortools.linear_solver import pywraplp

# Diet Problem
Given a variety type of food, how to minimize cost while satisfying all the nutritional content.


## Data
* FOOD_TB - Cost and min and max amount of each type of food
* AMT_TB - The nutritional value for each type of food
* NUT_TB - The min and max requirement for each type of nutriotional value* 

In [34]:
food_df = pd.read_csv('https://github.com/georgian-io/constrained_optimization_workshop/raw/main/FOOD_TB.csv', 
                      keep_default_na=False)
amt_df = pd.read_csv('https://github.com/georgian-io/constrained_optimization_workshop/raw/main/AMT_TB.csv', 
                      keep_default_na=False)
nut_df = pd.read_csv('https://github.com/georgian-io/constrained_optimization_workshop/raw/main/NUT_TB.csv', 
                      keep_default_na=False)

In [4]:
food_df

,FOOD,cost,f_min,f_max
0,BEEF,3.19,2,10
1,CHK,2.59,2,10
2,FISH,2.29,2,10
3,HAM,2.89,2,10
4,MCH,1.89,2,10
5,MTL,1.99,2,10
6,SPG,1.99,2,10
7,TUR,2.49,2,10


In [ ]:
amt_df

In [35]:
nut_df

,NUT,n_min,n_max
0,A,700,20000
1,B1,700,20000
2,B2,700,20000
3,C,700,20000
4,CAL,0,50000
5,NA,16000,24000


In [41]:
num_food = food_df.shape[0]
num_nut = nut_df.shape[0]

food_types = food_df['FOOD']
nut_types = nut_df['NUT']

food_cost = food_df.set_index(['FOOD'])['cost'].to_dict()
food_min = food_df.set_index(['FOOD'])['f_min'].to_dict()
food_max = food_df.set_index(['FOOD'])['f_max'].to_dict()
nut_min = nut_df.set_index(['NUT'])['n_min'].to_dict()
nut_max = nut_df.set_index(['NUT'])['n_max'].to_dict()

nut_amt = amt_df.set_index(['FOOD', 'NUT'])['amt'].to_dict()

# Define Solver

In [59]:
solver = pywraplp.Solver.CreateSolver('SCIP')

## Variables

x_i = number of food type i in diet

In [60]:
x = {}
for i,food in enumerate(food_types):
  x[food] = solver.IntVar(food_min[food],food_max[food], (str(food)))

## Constraints

In [61]:
# nutritional amount limit
for nut in nut_types:
  solver.Add(solver.Sum([x[food] * nut_amt[food,nut] for food in food_types]) >= nut_min[nut])
  solver.Add(solver.Sum([x[food] * nut_amt[food,nut] for food in food_types]) <= nut_max[nut])


## Objective Function

In [62]:
objective_terms = []
for food in food_types:
  objective_terms.append(x[food] * food_cost[food])

## Solve

In [63]:
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()

## Print Solution

In [69]:
print('Total Cost: ' + str(solver.Objective().Value()))
print('\nFood Amount: ')
for food in food_types:
  print(food + ': ' + str(int(x[food].solution_value())))

print('\nNutritional Values: ')
for nut in nut_types:
  print(nut + ': ' + str(int(sum([x[food].solution_value() * nut_amt[food,nut] for food in food_types]))))

Total Cost: 119.3

Food Amount: 
BEEF: 9
CHK: 2
FISH: 2
HAM: 8
MCH: 10
MTL: 10
SPG: 7
TUR: 2

Nutritional Values: 
A: 2037
B1: 1560
B2: 945
C: 700
CAL: 49793
NA: 19155
